# Semester 1 Project Submission



## Elliot

### The Movie Database Data Cleanse

In [5]:
#import needed libraries
import pandas as pd

In [6]:
#read tmdb file
tmdb_movie_db = pd.read_csv('databases/tmdb.movies.csv', encoding = 'utf-8', index_col = 0)
#drop extra title column
tmdb_movie_db = tmdb_movie_db.drop('original_title', axis = 1)
#standardize release date into date format
tmdb_movie_db['release_date'] = pd.to_datetime(tmdb_movie_db['release_date'])
#strip whitespace from title
tmdb_movie_db['title'] = tmdb_movie_db['title'].str.strip()
#remove duplicates
tmdb_movie_db = tmdb_movie_db.drop_duplicates()

tmdb_movie_db

,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,0.600,2018-06-22,Trailer Made,0.0,1


In [7]:
tmdb_movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25497 entries, 0 to 26516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genre_ids          25497 non-null  object        
 1   id                 25497 non-null  int64         
 2   original_language  25497 non-null  object        
 3   popularity         25497 non-null  float64       
 4   release_date       25497 non-null  datetime64[ns]
 5   title              25497 non-null  object        
 6   vote_average       25497 non-null  float64       
 7   vote_count         25497 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.8+ MB


In [8]:
#check to see that there are no duplicates
tmdb_movie_db.duplicated().value_counts() 

False    25497
dtype: int64

In [9]:
tmdb_movie_db.shape

(25497, 8)

### The Numbers Data Cleanse

In [48]:
#read tn file
tn_movie_db = pd.read_csv('databases/tn.movie_budgets.csv', encoding = 'utf-8', index_col = 0)
#standardize release date into date format
tn_movie_db['release_date'] = pd.to_datetime(tn_movie_db['release_date'])
#strip whitespace from title
tn_movie_db['movie'] = tn_movie_db['movie'].str.strip()
#strip $
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].str.strip('$')
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].str.strip('$')
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].str.strip('$')
#remove ,
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].replace(",", "", regex=True)
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].replace(",", "", regex=True)
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].replace(",", "", regex=True)
#convert to integers
tn_movie_db['production_budget'] = tn_movie_db['production_budget'].astype(int)
tn_movie_db['domestic_gross'] = tn_movie_db['domestic_gross'].astype(int)
tn_movie_db['worldwide_gross'] = tn_movie_db['worldwide_gross'].astype(float)
#create metric ratios
tn_movie_db['ww_gross / budget'] = tn_movie_db['worldwide_gross'] 

tn_movie_db['dom_gross / budget'] = tn_movie_db['domestic_gross']

tn_movie_db

,release_date,movie,production_budget,domestic_gross,worldwide_gross,ww_gross / budget,dom_gross / budget
id,,,,,,,
1,2009-12-18,Avatar,425000000,760507625,2.776345e+09,2.776345e+09,760507625
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1.045664e+09,1.045664e+09,241063875
3,2019-06-07,Dark Phoenix,350000000,42762350,1.497624e+08,1.497624e+08,42762350
4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1.403014e+09,1.403014e+09,459005868
5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1.316722e+09,1.316722e+09,620181382
...,...,...,...,...,...,...,...
78,2018-12-31,Red 11,7000,0,0.000000e+00,0.000000e+00,0
79,1999-04-02,Following,6000,48482,2.404950e+05,2.404950e+05,48482
80,2005-07-13,Return to the Land of Wonders,5000,1338,1.338000e+03,1.338000e+03,1338


In [33]:
tn_movie_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   release_date        5782 non-null   datetime64[ns]
 1   movie               5782 non-null   object        
 2   production_budget   5782 non-null   object        
 3   domestic_gross      5782 non-null   object        
 4   worldwide_gross     5782 non-null   object        
 5   ww_gross / budget   5782 non-null   object        
 6   dom_gross / budget  5782 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 361.4+ KB


In [12]:
#check to see that there are no duplicates
tn_movie_db.duplicated().value_counts() 

False    5782
dtype: int64

In [13]:
tn_movie_db.shape

(5782, 7)

## Ian

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Frida

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## Enrique

In [ ]:
# Your code here - remember to use markdown cells for comments as well!

## John

In [ ]:
# Your code here - remember to use markdown cells for comments as well!